<a href="https://colab.research.google.com/github/Jo-won/PatternRecognition_Term_2019Fall/blob/master/Level0_strongFeature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Downgrade cv
!yes | pip uninstall opencv-python
!yes | pip uninstall opencv-contrib-python
!yes | pip install opencv-python==3.4.2.16
!yes | pip install opencv-contrib-python==3.4.2.16
!pip install kmc2
! pip install cupy-cuda100

#mount
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Kaggle API Renewal. It should be version 1.5.6

!ls -lha kaggle.json
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle -v


In [0]:
import os
import glob
import cv2
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC, SVC
from sklearn.cluster import KMeans
import copy
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
import pandas as pd
import kmc2
from sklearn.cluster import MiniBatchKMeans


In [4]:
! ls

train_background = False
use_precompute = True
datasetPath = "path"
datasetTrainPath = os.path.join(datasetPath + 'train')
datasetTestPath = os.path.join(datasetPath + 'testAll_v2')
className = os.listdir(datasetTrainPath)
className.sort()
classNameDict = dict(zip(range(0,len(className)), className))


drive  kaggle.json  sample_data


In [0]:
# load training dataset

trainDataset = []
trainLabel = []

lab = -1
for i, name in tqdm.tqdm(enumerate(className)):

    if className[i-1] != className[i]:
        lab+=1
    filename = os.listdir(os.path.join(datasetTrainPath+'/'+name))
    
    if train_background is False: # except background
    
        if i==0:
            continue
    
    for img in filename:

        if use_precompute is False:
            data = cv2.resize(cv2.imread(datasetTrainPath+'/'+name+'/'+img), (256,256), interpolation = cv2.INTER_LINEAR)
            data = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
            trainDataset.append(data)

        trainLabel.append(lab)

if use_precompute is False:

    trainDataset = np.asarray(trainDataset)
    print(trainDataset.shape)

trainLabel = np.asarray(trainLabel)
print(trainLabel.shape)

In [0]:
# load test dataset

testDataset = []
filename = os.listdir(os.path.join(datasetTestPath))
filename.sort()

for img in tqdm.tqdm(filename):

    if use_precompute is False:
        data = cv2.resize(cv2.imread(datasetTestPath+'/'+img), (256,256), interpolation = cv2.INTER_LINEAR)
        data = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
        testDataset.append(data)

if use_precompute is False:
    testDataset = np.asarray(testDataset)
    print(testDataset.shape)

In [0]:
# compute dense SIFT 
def computeSIFT(data):
    x = []
    for i in tqdm.tqdm(range(0, len(data)), desc = "Processing"):
        sift = cv2.xfeatures2d.SIFT_create()
        img = data[i]
        step_size = 8
        kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, img.shape[0], step_size) for y in range(0, img.shape[1], step_size)]
        dense_feat = sift.compute(img, kp)
        
        x.append(dense_feat[1])
        
    return x

In [0]:
feats_save_path = 'path'

if os.path.isfile():
    savez_load = np.load(feats_save_path + 'feats_save.npz')
    trainFeats = savez_load['train']
    validFeats = savez_load['valid']
    testFeats = savez_load['test']
    trainLabel = 
else:
    trainFeats = computeSIFT(trainDataset)
    validFeats = computeSIFT(validDataset)
    testFeats = computeSIFT(testDataset)
    np.savez(feats_save_path , train=trainFeats, valid=validFeats, test=testFeats)

In [0]:
codebooksize = 200
seeding = kmc2.kmc2(np.array(trainFeats).reshape(-1,128), codebooksize)
kmeans = KMeans(codebooksize, init=seeding).fit(np.array(trainFeats).reshape(-1,128))

In [0]:
codebook = kmeans.cluster_centers_

In [0]:
codeword_save_path = "경로"
print(codebook.shape)
print(codebook)
np.savez(codeword_save_path + 'codeword_kmeans', codeword=codeword)

In [0]:
use_vq = True

In [0]:
if use_vq is True:
    from scipy.cluster.vq import vq
    def input_vector_encoder(feature, codebook):
        code, _ = vq(feature, codebook)
        word_hist, bin_edges = np.histogram(code, bins=range(codebook.shape[0] + 1), normed=True)
        return word_hist

    def make_hist(feature, codebook):
        train_hist = []
        
        for i in tqdm.tqdm(range(len(feature))):
            
            data = copy.deepcopy(feature[i])
            predict = input_vector_encoder(data, codebook)
            train_hist.append(predict)
            
        return np.array(train_hist)
        
    train_hist = make_hist(trainFeats, codebook )
    valid_hist = make_hist(validFeats, codebook )
    test_hist = make_hist(testFeats, codebook )

else:
    def formTrainingSetHistogram(x_train, kmeans, k):
        train_hist = []
        for i in tqdm.tqdm(range(len(x_train))):
            data = copy.deepcopy(x_train[i])
            predict = kmeans.predict(data)
            
            train_hist.append(np.bincount(predict, minlength=k).reshape(1,-1).ravel())
            
        return np.array(train_hist)
    # form training and testing histograms
    train_hist = formTrainingSetHistogram(trainFeats, kmeans, codebooksize )
    valid_hist = formTrainingSetHistogram(validFeats, kmeans, codebooksize )
    test_hist = formTrainingSetHistogram(testFeats, kmeans,codebooksize )

In [0]:
# normalize histograms
scaler = preprocessing.StandardScaler().fit(train_hist)
train_hist_ = scaler.transform(train_hist)
valid_hist_ = scaler.transform(valid_hist)
test_hist_ = scaler.transform(test_hist)

In [0]:
model = SVC(random_state=0, C=5) # or C=100
model.fit(train_hist_, trainLabel)

In [0]:
train = model.predict(train_hist_)
print ("Accuracy:", np.mean(train == trainLabel)*100, "%")

In [0]:
predict = model.predict(test_hist_)

In [0]:
df = pd.DataFrame({"Id":filename,"Category":predict})
df.index = np.arange(1,len(df)+1)
df.to_csv('./results_191209_26.csv',index=False, header=True)